This is a case of Image data augmentation.
Image data augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset.

Training deep learning neural network models on more data can result in more skillful models, and the augmentation techniques can create variations of the images that can improve the ability of the fit models to generalize what they have learned to new images.

The Keras deep learning neural network library provides the capability to fit models using image data augmentation via the ImageDataGenerator class.
 how to improve image classification
by using data augmentation and convolutional neural networks. Model
overfitting and poor performance are common problems in applying neural network techniques. Approaches to bring intra-class differences down
and retain sensitivity to the inter-class variations are important to maximize model accuracy and minimize the loss function. The image dataset, the effects of model overfitting were monitored
within different model architectures in combination of data augmentation and hyper-parameter tuning. The model performance was evaluated
with train and test accuracy and loss, characteristics derived from the
confusion matrices, and visualizations of different model outputs. As a macro-architecture with 500 weighted layers, 
model is used for large scale image classification. In the presence of image
data augmentation, the overall model train accuracy is 96%, the
test accuracy is stabilized at 92%, and both the results of train and test
losses are below 0.5. The overall image classification error rate is dropped
to 8%, while the single class misclassification rates are less than 7.5% in
eight out of ten image classes. Model architecture, hyper-parameter tuning, and data augmentation are essential to reduce model overfitting and
help build a more reliable convolutional neural network model.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import pydicom
from skimage.transform import resize


Reading the csv to fetch the data and 

In [ ]:
submission_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv', index_col=None)
image_df = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv', index_col=None)
study_df = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv', index_col=None)

print(f"Train image level csv shape : {image_df.shape}")
print(f"Train study level csv shape : {study_df.shape}")

In [ ]:
import matplotlib.pyplot as plt

# Data to plot
labels = 'Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance'
sizes = [study_df['Negative for Pneumonia'].sum(), study_df['Typical Appearance'].sum(), study_df['Indeterminate Appearance'].sum(), study_df['Atypical Appearance'].sum()]
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
explode = (0.1, 0, 0, 0)  # explode 1st slice

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()

From the above plot, it is clear that the typical appearance has highest number at the same time. Atypical Appearance has the lowest percentage

In [ ]:
study_df

In [ ]:
submission_df

In [ ]:
image_df

In [ ]:
i=0
j=0
k=0
l=0
m=0
train_path=[]
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/train/'):
    for filename in filenames:
        str_split =filename.split('.')
        if str_split[1]=="dcm" :
            df=image_df[image_df['id'].str.contains(str_split[0])]
            studydf=study_df[study_df.id.isin(df.StudyInstanceUID+'_study')]
            lbl_folder=studydf.columns[studydf.eq(1).any()] 
            if lbl_folder=='Atypical Appearance':
                if i<10:
                    train_path.append(os.path.join(dirname, filename))
                    i=i+1
            elif lbl_folder=='Indeterminate Appearance':
                if j<10:
                    train_path.append(os.path.join(dirname, filename))
                    j=j+1
            elif lbl_folder=='Negative for Pneumonia':
                if k<10:
                    train_path.append(os.path.join(dirname, filename))
                    k=k+1
            elif lbl_folder=='Typical Appearance':
                if l<10:
                    train_path.append(os.path.join(dirname, filename))
                    l=l+1
            else :
                if m<10:
                    train_path.append(os.path.join(dirname, filename))
                    m=m+1

In [ ]:
for rows in train_path:
    # specify your image path
    try:
        ct_dicom = pydicom.read_file(rows)
        plt.imshow(ct_dicom.pixel_array, cmap='gray')
        plt.show()
    except:
          pass
    
    
  

In [ ]:
import cv2
import os
from skimage.transform import resize

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.patches as patches
dsimagedetails=[]
box=[]
for rows in train_path:
    #rows="/kaggle/input/siim-covid19-detection/train/ea44061da219/bd68195b6b27/a818e98c3f90.dcm"
    
    imgname=rows.rsplit('/', 1)[1]
    str_split =imgname.split('.')
    imagename=str_split[0]
    img_size = 224
    data=[]
    df=image_df[image_df['id'].str.contains(imagename,regex=True)]
    studydf=study_df[study_df.id.isin(df.StudyInstanceUID+'_study')]
    imgclass=studydf.columns[studydf.eq(1).any()]
    data_items = df['boxes'].items()
    data_list = list(data_items)
    boxes_df = pd.DataFrame(data_list)
    if boxes_df.empty:
        print('DataFrame is empty!')
    else:
        boxes_df[1].fillna(0, inplace=True)
        if boxes_df[1].get(0) != 0:
            ss=boxes_df[1].get(0).split('},')
            x1=float(ss[0].split(',')[0].split(':')[1].strip())
            y1=float(ss[0].split(',')[1].split(':')[1].strip())
            width1=float(ss[0].split(',')[2].split(':')[1].strip())
            if '}]' in ss[0].split(',')[3].split(':')[1].strip() :
                height1=float(ss[0].split(',')[3].split(':')[1].strip().replace("}]", ""))
                box = [
                       {'x' : x1, 'y' : y1, 'width' : width1, 'height' : height1},
                      ]
                dsimagedetails.append({'data':rows,'label' : imgclass[0],'x' : x1, 'y' : y1, 'width' : width1, 'height' : height1 })
              
            else:
                height1=float(ss[0].split(',')[3].split(':')[1].strip()) 
                x2=float(ss[1].split(',')[0].split(':')[1].strip())
                y2=float(ss[1].split(',')[1].split(':')[1].strip())
                width2=float(ss[1].split(',')[2].split(':')[1].strip())
                height2=float(ss[1].split(',')[3].split(':')[1].replace('}]', '').replace("'", "").strip())
                box = [
                      {'x' : x1, 'y' : y1, 'width' : width1, 'height' : height1},
                      {'x' : x2, 'y' : y2, 'width' : width2, 'height' : height2},
                    ]
                dsimagedetails.append({'data':rows,'label' : imgclass[0],'x1' : x1, 'y1' : y1,'x' : x2, 'y' : y2, 'width' : width1, 'height' : height1 })
        dfbox= pd.DataFrame(box)
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    dicom_image = pydicom.read_file(rows) 
    try:
        img = dicom_image.pixel_array
                #print(path+imagename+'.jpg')
              #scipy.misc.imsave (path+imagename+'.jpg', img)
        a.imshow(img, cmap = 'gray')
        for index, row in dfbox.iterrows(): 
            x, y, width, height  = row['x'], row['y'], row['width'], row['height']
            rect = patches.Rectangle((x, y),
                                     width, height,
                                     linewidth = 2,
                                     edgecolor = 'r',
                                     facecolor = 'none')

                # Draw the bounding box on top of the image
            a.add_patch(rect)
        plt.show()
    except:
        pass
    

In [ ]:
dsimg=[]
labels = ['Atypical Appearance', 'Indeterminate Appearance', 'Negative for Pneumonia', 'Typical Appearance', 'Not Identified']
img_size = 224
for rows in dsimagedetails:
    ds= pydicom.dcmread(rows['data'])
    img_arr = ds.pixel_array
    resized_arr = resize(img_arr, (224, 224), anti_aliasing=True)[...,::-1] # Reshaping images to preferred size
    bb=np.array([rows['x'],row['y'],row['width'],row['height']])
    for lrow in labels:
        if rows['label']==lrow:
            classnum=labels.index(label)
        else:
            classnum=4
    dsimg.append(resized_arr,classnum,bb)
    
        
    

In [ ]:
# define the amount of data that will be used training
TRAIN_SPLIT = 0.5
# the amount of validation data will be a percentage of the
# *training* data
VAL_SPLIT = 0.5
# import the necessary packages
import random
import shutil
import os
# grab the paths to all input images in the original input directory
# and shuffle them
random.seed(42)
random.shuffle(dsimg)
# compute the training and testing split
i = int(len(dsimg) * TRAIN_SPLIT)
trainds = dsimg[:i]
valds = dsimg[i:]

In [ ]:
import seaborn as sns
from tensorflow import keras 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf


In [ ]:
trainds

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label,bb in trainds:
    x_train.append(feature)
    y_train.append(label)
    z_train.append(bb)
for feature, label in valds:
    x_val.append(feature)
    y_val.append(label)
    z_val.append(bb)
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train =x_train.reshape(-1, 224, 224, 1)
y_train=y_train = np.array(y_train)
z_train=z_train = np.array(z_train)

x_val =x_val.reshape(-1, 224, 224, 1)
y_val = np.array(y_val)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,1)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(2048,activation="relu"))
model.add(Dense(4, activation="softmax"))

model.build(input_shape=(224,224,3))
model.summary()

In [ ]:
opt = Adam(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [ ]:
print(x_val.shape)
print(x_train.shape)

In [ ]:
history = model.fit(x_train,y_train,epochs = 50 , validation_data = (x_val,y_val))


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
predictions = model.predict_classes(x_val)
predictions = predictions.reshape(1,-1)[0]
print(predictions)

In [ ]:
print(classification_report(y_val, predictions, target_names =['Atypical Appearance (Class 0)', 'Indeterminate Appearance (Class 1)', 'Negative for Pneumonia (Class 2)',  'Typical Appearance (Class 4)']))

Predicting the classes for the test data whose label are not given

In [ ]:
test_path=[]
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/test/'):
    for filename in filenames:
        str_split =filename.split('.')
        if str_split[1]=="dcm" :
            test_path.append(os.path.join(dirname, filename))


In [ ]:
random.shuffle(test_path)

In [ ]:
testpath=[]
i=0
for rows in test_path:
    if i<51:
        testpath.append(rows)
        i=i+1

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
ds_finalsubmission=[]
for rows in test_path:        
    try:
        img=rows.rsplit('/', 1)[1]
        img=img.split('.')
        img=img[0]+'_study'
        dstest= pydicom.dcmread(rows)
        img_arrtest = dstest.pixel_array
        resized_arrtest = resize(img_arrtest, (224, 224), anti_aliasing=True)
        x_test = np.array(resized_arrtest) / 255
        x_test =x_test.reshape(-1, 224, 224, 1)
        predictions = probability_model.predict(x_test)
        predicted_label = np.argmax(predictions)
        
        if predicted_label==0:
            title='atypical 1 0 0 1 1'
        elif predicted_label==1:
            title='intermediate 1 0 0 1 1'
        elif predicted_label==2:
            title='negative 1 0 0 1 1'
        elif predicted_label==3:
            title='typical 1 0 0 1 1'
        elif predicted_label==4:
            img=img[0]+'_image'
            title='none 1 0 0 1 1'
        ds_finalsubmission.append(img,title)
    except: 
        pass
               


In [ ]:
df_finalsubmission=pd.DataFrame(ds_finalsubmission,columns = ["Id", "PredictionString"])

In [ ]:
df_finalsubmission

In [ ]:
df_finalsubmission.to_csv('submission.csv',index=False)
